In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Library Import

## Description
This code segment imports essential libraries for data manipulation, deep learning operations, and URL handling.

## Libraries
- `csv`: For CSV file I/O.
- `tensorflow`: Deep learning framework.
- `numpy`: Numerical computation library.
- `urllib`: URL handling library.
- `pandas`: Data manipulation and analysis library.
- `datetime` : Class for manipulating date and time

In [ ]:
import csv
import tensorflow as tf
import numpy as np
import urllib
import pandas as pd
from datetime import datetime

# Windowed Dataset Function

## Description
This function prepares a windowed dataset from a time series for training recurrent neural networks.

## Parameters
- `series`: Time series data.
- `window_size`: Size of the sliding window.
- `batch_size`: Batch size for training.
- `shuffle_buffer`: Buffer size for shuffling data.

## Functionality
- Expands dimensions of the series.
- Creates a dataset from tensor slices.
- Divides the series into windows with specified size and shift.
- Flattens the dataset and maps batches.
- Shuffles the dataset and maps input-output pairs.
- Batches and prefetches the dataset for training.


In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

# Reading Time Series Data

## Description
This code segment reads time series data from a CSV file and prepares it for analysis.

## Parameters
- `data_path`: Path to the CSV file containing time series data.

## Functionality
- Reads the CSV file and extracts temperature data.
- Converts date strings to datetime objects.
- Prepares time steps and temperature values for further analysis.


In [ ]:
data_path = '/content/drive/MyDrive/GSC/Time Series Model/data_iklim_harian.csv'

time_step = []
suhu = []

with open(data_path) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        date_str = row[0]
        suhu.append(float(row[1]))
        date_obj = datetime.strptime(date_str, '%d-%m-%Y')
        time_step.append(date_obj)

series = np.array(suhu)


# Data Normalization and Windowed Dataset Preparation

## Description
This code segment normalizes the time series data and prepares windowed datasets for training and validation.

## Data Normalization
- Computes the minimum and maximum values of the series.
- Normalizes the series to the range [0, 1].

## Time Series Splitting
- Splits the time series and corresponding data into training and validation sets.

## Windowed Dataset Preparation
- Defines parameters for creating windowed datasets, including window size, batch size, and shuffle buffer size.
- Prepares a windowed dataset for training using the `windowed_dataset` function.


In [ ]:
min = np.min(series)
max = np.max(series)
series -= min
series /= max
time = np.array(time_step)

split_time = 300

time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]


window_size = 30
batch_size = 32
shuffle_buffer_size = 100

train_set = windowed_dataset(
    x_train,
    window_size=window_size,
    batch_size=batch_size,
    shuffle_buffer=shuffle_buffer_size
    )

# Time Series Forecasting Model

## Description
This code defines a deep learning model for time series forecasting using a combination of convolutional and recurrent neural network layers.

## Model Architecture
- `Sequential Model`: Sequentially stacks layers.
- `Conv1D Layer`: Convolutional layer with 64 filters, kernel size 3, ReLU activation, and causal padding.
- `LSTM Layers`: Three LSTM layers with 64 units each, return sequences for the first two layers, and dropout regularization.
- `Dense Layers`: Fully connected layers with ReLU activation.
- `BatchNormalization Layer`: Normalizes the activations of the previous layer.
- Output Layer: Dense layer with linear activation for forecasting.

## Input Shape
- Input shape: [None, 1] (variable length sequence, single feature).

## Regularization
- `Dropout Layers`: Applied after LSTM layers for regularization.

## Output
- Predicts a single value for each time step.

## Note
- Adjust the architecture and hyperparameters based on the specific requirements and characteristics of the time series data.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='causal', input_shape=[None, 1]),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

# Learning Rate Schedule and Model Compilation

## Description
This code defines a learning rate schedule and compiles the time series forecasting model with a specified optimizer, loss function, and evaluation metrics.

## Learning Rate Schedule
- `ExponentialDecay`: Exponential decay learning rate schedule.
  - `initial_learning_rate`: Initial learning rate (1e-3).
  - `decay_steps`: Number of steps for decay (10000).
  - `decay_rate`: Rate of decay (0.9).

## Optimizer
- `Adam Optimizer`: Adaptive learning rate optimizer with the specified learning rate schedule.

## Model Compilation
- `Huber Loss`: Robust loss function less sensitive to outliers.
- `Metrics`: Mean absolute error (MAE) as the evaluation metric.

## Note
- Adjust the learning rate schedule, optimizer, loss function, and evaluation metrics based on the specific requirements and characteristics of the time series forecasting task.


In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=optimizer,
    metrics=['mae']
)

In [ ]:
model.fit(train_set, epochs=100)

Epoch 1/100
9/9 [==============================] - 9s 89ms/step - loss: 0.0032 - mae: 0.0659
Epoch 2/100
9/9 [==============================] - 1s 56ms/step - loss: 0.0013 - mae: 0.0414
Epoch 3/100
9/9 [==============================] - 1s 54ms/step - loss: 9.4549e-04 - mae: 0.0346
Epoch 4/100
9/9 [==============================] - 1s 57ms/step - loss: 7.2845e-04 - mae: 0.0307
Epoch 5/100
9/9 [==============================] - 1s 55ms/step - loss: 6.0350e-04 - mae: 0.0279
Epoch 6/100
9/9 [==============================] - 1s 54ms/step - loss: 5.3961e-04 - mae: 0.0263
Epoch 7/100
9/9 [==============================] - 1s 56ms/step - loss: 5.5491e-04 - mae: 0.0269
Epoch 8/100
9/9 [==============================] - 1s 53ms/step - loss: 4.9342e-04 - mae: 0.0251
Epoch 9/100
9/9 [==============================] - 1s 56ms/step - loss: 4.9911e-04 - mae: 0.0251
Epoch 10/100
9/9 [==============================] - 1s 54ms/step - loss: 4.6825e-04 - mae: 0.0249
Epoch 11/100
9/9 [===================

In [ ]:
model.save('/content/drive/MyDrive/GSC/Time Series Model/time_series_model')

# Time Series Forecasting Function

## Description
This function performs time series forecasting using the trained model.

## Parameters
- `model`: Trained time series forecasting model.
- `series`: Time series data.
- `window_size`: Size of the sliding window for input sequences.
- `days`: Number of days for forecasting.

## Functionality
- Iterates through the time series data to generate initial forecasts.
- Extends the forecast for the specified number of days using the model's predictions.
- Rescales the forecasted values to the original scale.
- Generates a list of forecasted temperatures for each future date.

## Output
- List of dictionaries containing forecasted temperatures for each future date.

## Note
- Ensure that the model, series, window size, and number of days are properly configured before calling this function.


In [ ]:
from datetime import datetime, timedelta
def forecasting(model, series, window_size, days):
    forecast = []
    for time in range(len(series) - window_size):
        forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

    for _ in range(days):
        prediction = model.predict(series[-window_size:][np.newaxis])[0,0]
        series = np.append(series, prediction)
        forecast.append(prediction)

    for i in range(len(forecast)):
        if not isinstance(forecast[i], np.ndarray):
            forecast[i] = np.array([[forecast[i]]])

    forecast = np.array(forecast) * max + min

    result = []
    for i in range(days):
        next_date = time_step[-1] + timedelta(days=i+1)
        result.append({'day': next_date.strftime('%d-%m-%Y'), 'temperature': forecast[-days + i]})

    return result

forecast_result = forecasting(model, series, window_size, days = 7)
print(forecast_result)

1/1 [==============================] - 0s 28ms/step
[{'day': '01-01-2024', 'temperature': array([[28.025572]], dtype=float32)}, {'day': '02-01-2024', 'temperature': array([[27.994965]], dtype=float32)}, {'day': '03-01-2024', 'temperature': array([[28.008997]], dtype=float32)}, {'day': '04-01-2024', 'temperature': array([[28.040058]], dtype=float32)}, {'day': '05-01-2024', 'temperature': array([[28.061123]], dtype=float32)}, {'day': '06-01-2024', 'temperature': array([[28.081366]], dtype=float32)}, {'day': '07-01-2024', 'temperature': array([[28.09162]], dtype=float32)}]


In [ ]:
import tensorflow as tf
import numpy as np

def forecast_one_week(model, series, window_size, days):
    forecast = []
    for t in range(len(series) - window_size):
        forecast.append(model.predict(series[t:t + window_size][np.newaxis]))

    forecast = forecast[-days:]
    results = np.array(forecast)[:, 0, 0]

    return results

forecast = forecast_one_week(model=model, series=series, window_size=window_size, days=360)

forecast = forecast * max + min

for day, temp in enumerate(forecast, 1):
    print(f"Prediksi suhu untuk hari {day}: {temp:.2f} derajat Celcius")


1/1 [==============================] - 0s 26ms/step
Prediksi suhu untuk hari 1: 26.49 derajat Celcius
Prediksi suhu untuk hari 2: 26.47 derajat Celcius
Prediksi suhu untuk hari 3: 26.47 derajat Celcius
Prediksi suhu untuk hari 4: 26.48 derajat Celcius
Prediksi suhu untuk hari 5: 26.49 derajat Celcius
Prediksi suhu untuk hari 6: 26.49 derajat Celcius
Prediksi suhu untuk hari 7: 26.50 derajat Celcius
Prediksi suhu untuk hari 8: 26.51 derajat Celcius
Prediksi suhu untuk hari 9: 26.52 derajat Celcius
Prediksi suhu untuk hari 10: 26.53 derajat Celcius
Prediksi suhu untuk hari 11: 26.54 derajat Celcius
Prediksi suhu untuk hari 12: 26.56 derajat Celcius
Prediksi suhu untuk hari 13: 26.59 derajat Celcius
Prediksi suhu untuk hari 14: 26.60 derajat Celcius
Prediksi suhu untuk hari 15: 26.62 derajat Celcius
Prediksi suhu untuk hari 16: 26.65 derajat Celcius
Prediksi suhu untuk hari 17: 26.67 derajat Celcius
Prediksi suhu untuk hari 18: 26.70 derajat Celcius
Prediksi suhu untuk hari 19: 26.73 dera